# IBM Data Science Professional Certificate: Coursera Capstone

## Week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Part 1 - Scraping Toronto neighborhood data from Wikipedia

First install necessary packages/libraries:

In [2]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 5.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


Create a url variable from wikipedia page and use this to create html_data variable:

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

Create Beautiful Soup object using the html_data variable:

In [4]:
soup = BeautifulSoup(html_data,"html5lib")

Scrape the soup object through following steps:

+ Create a list called table_contents
+ Find the table within the soup object
+ Create dictionary called cell
+ Pass any cells in the table that contain 'Not assigned' for neighborhood values
+ Extract Postal Code from first table cell paragraph by selecting the the first 3 characters, put it into the cell dictionary that was created under the value 'PostalCode'
+ Extract Borough by scanning through the remaining text of the cell, splitting it by parenthesis and taking the first part of the split, put it into cell dict under value 'Borough'
+ Extract Neighborhood by scanning through the remaining text of the cell, splitting it by parenthesis and taking the second part of the split
+ From this part of the split, strip the second parenthesis, replace any forward slashes with commas, replace extra closing parentheses with spaces and finally strip extra spaces
+ Put this result into cell dict under value 'Neighborhood'
+ Append the values currently stored in the cell dict to the table_contents list
+ Move on to next cell in the table
+ Print the table_contents list which contains all the dict values that were stored

In [5]:
table_contents=[]
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if  row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

Convert the list of dictionaries into a pandas dataframe:

In [6]:
toronto_addresses=pd.DataFrame(table_contents)
toronto_addresses

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Simplify complicated borough names:

In [7]:
toronto_addresses['Borough']=toronto_addresses['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_addresses

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Print shape of cleaned dataframe:

In [8]:
print(toronto_addresses.shape)

(103, 3)


### Part 2 - Adding latitude and longitude coordinates

Read in coordinate data from Geospatial data csv file to a new dataframe:

In [9]:
toronto_coords = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_coords.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Join address data with coordinate data into new dataframe:

In [10]:
toronto_locations = toronto_addresses.join(toronto_coords, on='PostalCode')
toronto_locations.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Part 3 - Exploring and clustering the neighborhoods

Import necessary packages:

In [11]:
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarc

Use geolocator to source the coordinates of Toronto:

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Use Folium to create map of Toronto then add markers to map based on the neighborhoods stored in the toronto_locations dataframe:

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_locations['Latitude'], toronto_locations['Longitude'], toronto_locations['Borough'], toronto_locations['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create new dataframe by slicing only boroughs that contain the word 'Toronto':

In [14]:
toronto_borough_locations = toronto_locations[toronto_locations.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_borough_locations

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


Create new map using the new dataframe that has only boroughs containing 'Toronto'

In [15]:
map_toronto_borough = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_borough_locations['Latitude'], toronto_borough_locations['Longitude'], toronto_borough_locations['Borough'], toronto_borough_locations['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_borough)  
    
map_toronto_borough

#### Explore the neighborhoods using Foursquare

Import json packages:

In [16]:
import json
from pandas.io.json import json_normalize

Store Foursquare credentials and version:

In [17]:
CLIENT_ID = '3QWZRJM0J0CXOHIFAPXYG2IVAKTA43M0FJIDA5ZRHND0HVRE'
CLIENT_SECRET = '5SKV3M4TPP44PYCWASQZNAQQPQXJ1KKSF5K0H4WFHAAH24FB'
VERSION = '20180605'
LIMIT = 100

First define a function to cycle through each Toronto neighborhood using the explore request of the Foursquare API, sourcing information about each:

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now use the function to get nearby venues for each Toronto neighborhood:

In [19]:
toronto_venues = getNearbyVenues(names=toronto_borough_locations['Neighborhood'], latitudes=toronto_borough_locations['Latitude'], longitudes=toronto_borough_locations['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Check how many venues are stored for each neighborhood:

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33
Davisville North,6,6,6,6,6,6


Find out how many unique categories there are:

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


#### Analyse the neighborhoods

Use one hot encoding to determine the frequency of occurance of each venue category within each neighborhood:

In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 235)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Define function that sorts venues into descending order:

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create new dataframe with top 10 venues for each neighborhood based on frequency:

In [86]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Bar,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Plane,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Korean Restaurant,Indian Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Candy Store


#### Cluster the neighborhoods

Set number of clusters and use _k_-means to cluster the neighborhoods, then check the cluster labels:

In [87]:
kclusters = 8
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0], dtype=int32)

Create new dataframe that combines the original location data for toronto boroughs only with the cluster labels and most common venues:

In [88]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_locations_clusters_venues = toronto_borough_locations
toronto_locations_clusters_venues = toronto_locations_clusters_venues.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_locations_clusters_venues.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Hotel,Ramen Restaurant,Pizza Place,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,Restaurant,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Health Food Store,Trail,Pub,Wings Joint,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant


Create folium map that visualises the resulting clusters:

In [89]:
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    toronto_locations_clusters_venues['Latitude'], 
    toronto_locations_clusters_venues['Longitude'], 
    toronto_locations_clusters_venues['Neighborhood'], 
    toronto_locations_clusters_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

#### Examine the clusters

From cycling through different numbers of clusters a suitable value for _k_ looks to be 8.
Looking at each cluster individually can help to distinguish the characteristics that are unique to each:

##### Cluster one:

In [90]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 0, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Candy Store
8,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pet Store,Music Venue,Middle Eastern Restaurant,Bar,Café,Furniture / Home Store,Bank,Supermarket
14,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
27,"University of Toronto, Harbord",Café,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Sandwich Place,Yoga Studio,Dessert Shop,Comfort Food Restaurant


Based on the above, cluster one could be labelled 'University and sports facilities areas'

##### Cluster two:

In [91]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 1, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Hotel,Ramen Restaurant,Pizza Place,Burger Joint
2,St. James Town,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,Restaurant,Italian Restaurant
4,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant
5,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Korean Restaurant,Indian Restaurant
7,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Pizza Place
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,Scenic Lookout
11,"Little Portugal, Trinity",Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Café,Coffee Shop,Asian Restaurant,Men's Store,Yoga Studio,Miscellaneous Shop
12,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Yoga Studio,Pub,Indian Restaurant
13,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Italian Restaurant,Seafood Restaurant,Salad Place,Restaurant,Japanese Restaurant,Sushi Restaurant,Breakfast Spot


Based on the above, cluster two could be labelled 'Central & tourist hospitality areas'

##### Cluster three:

In [92]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 2, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Summer Camp,Tennis Court,Wings Joint,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Based on the above, cluster three could be labelled 'Summer camp area'

##### Cluster four:

In [93]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 3, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Rosedale,Park,Playground,Trail,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Based on the above, cluster four could be labelled 'City green space areas'

##### Cluster five:

In [94]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 4, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Garden,Wings Joint,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Based on the above, cluster five could be labelled 'Retail park area'

##### Cluster six:

In [95]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 5, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,Pizza Place,Park,Convenience Store,Intersection,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Based on the above, cluster six could be labelled 'Toronto / East York border'

##### Cluster seven:

In [97]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 6, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,Health Food Store,Trail,Pub,Wings Joint,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop


Based on the above, cluster seven could be labelled 'Beach district'

##### Cluster eight:

In [98]:
toronto_locations_clusters_venues.loc[toronto_locations_clusters_venues['Cluster Labels'] == 7, 
                                      toronto_locations_clusters_venues.columns[[2] + list(range(6, toronto_locations_clusters_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Business Service,Comic Shop,Concert Hall,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Based on the above, cluster eight could be labelled 'Toronto border park area'